In [4]:
# pip install praat-textgrids

In [1]:
import os
import random
from string import punctuation

from tqdm import tqdm

import transformers
import sentencepiece
import torchaudio
import textgrids

import numpy as np
import torch as th
import pandas as pd

from g2p_en import G2p

from fairseq.data import PhonemeDictionary
from ConST.prepare_data.data_utils import load_df_from_tsv, save_df_to_tsv

In [2]:
root = '/mnt/data/siqiouyang/datasets/must-c-v1.0/'
lang = 'es'

In [3]:
# spm = sentencepiece.SentencePieceProcessor(os.path.join(root, 'spm_unigram10000_st_{}.model'.format(lang)))
# spm = sentencepiece.SentencePieceProcessor(os.path.join(root, 'flores200sacrebleuspm.model'))
tokenizer = transformers.AutoTokenizer.from_pretrained(
    '/mnt/taurus/data/xixu/llm/llama-2-7b/hf',
    padding_side="right",
    use_fast=False,
)

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [41]:
split = 'train'

In [42]:
df = load_df_from_tsv(os.path.join(root, '{}_st_{}.tsv'.format(split, lang)))
# df = load_df_from_tsv(os.path.join(root, '{}_st_{}.tsv'.format(split)))
# df = load_df_from_tsv(os.path.join(root, 'train_asr_mt_iwslt.tsv'))
# df2 = load_df_from_tsv(os.path.join(root, 'dev_asr_mt_iwslt.tsv'))
# df3 = load_df_from_tsv(os.path.join(root, 'train_asr_mt_cv.tsv'))
# df4 = load_df_from_tsv(os.path.join(root, 'dev_asr_mt_cv.tsv'))
# df2 = load_df_from_tsv(os.path.join(root, 'dev_st_id_en.tsv'))
# df3 = load_df_from_tsv(os.path.join(root, 'test_st_id_en.tsv'))

In [43]:
# df = pd.concat([df, df2, df3, df4], ignore_index=True)

In [44]:
df

,id,audio,n_frames,speaker,src_text,tgt_text,src_lang,tgt_lang
0,ted_1_0,en-es/data/train/wav/ted_1.wav:1739840:130720,130720,spk.1,(Laughter) I looked in the rear-view mirror an...,"Sé que suena como cualquier cosa para ustedes,...",en,es
1,ted_1_1,en-es/data/train/wav/ted_1.wav:1872800:99679,99679,spk.1,There was no motorcade back there. (Laughter) ...,No había caravana de vehículos allá atrás. ¿Ha...,en,es
2,ted_1_2,en-es/data/train/wav/ted_1.wav:2163679:74880,74880,spk.1,"(Laughter) It was dinnertime, and we started l...","Era la hora de la cena, y comenzamos a buscar ...",en,es
3,ted_1_3,en-es/data/train/wav/ted_1.wav:2254400:8800,8800,spk.1,We were on I-40.,Estábamos en la I-40.,en,es
4,ted_1_4,en-es/data/train/wav/ted_1.wav:2271040:51840,51840,spk.1,"We got to Exit 238, Lebanon, Tennessee.","Llegamos a la Salida 238, Líbano, Tennessee.",en,es
...,...,...,...,...,...,...,...,...
260036,ted_13591_40,en-es/data/train/wav/ted_13591.wav:851680:126240,126240,spk.13591,Then I would beat my plate with a spoon and my...,Luego golpeaba mi plato con una cuchara y me b...,en,es
260037,ted_13591_41,en-es/data/train/wav/ted_13591.wav:988800:172160,172160,spk.13591,"I have been beating the same plates, shakers, ...",Desde entonces sigo golpeando los mismos plato...,en,es
260038,ted_13591_43,en-es/data/train/wav/ted_13591.wav:1263840:96160,96160,spk.13591,"As I grew up, subconsciously, I felt a strong ...","A medida que iba creciendo, sentía inconscient...",en,es
260039,ted_13591_44,en-es/data/train/wav/ted_13591.wav:1363680:91520,91520,spk.13591,"Even without it being said out loud, I knew th...","Y aunque no lo decía en voz alta, sabía que es...",en,es


In [45]:
df['n_frames'].sum() / 16000 / 3600

479.9419181770833

In [46]:
# n_frames = 0
# for fn in os.listdir('/mnt/data/siqiouyang/datasets/must-c-v1.0/en-de/data/dev/wav'):
#     info = torchaudio.info('/mnt/data/siqiouyang/datasets/must-c-v1.0/en-de/data/dev/wav/' + fn)
#     n_frames += info.num_frames
# n_frames / 16000 / 3600

# import yaml
# with open('/mnt/data/siqiouyang/datasets/must-c-v1.0/en-de/data/train/txt/train.yaml') as r:
#     y = yaml.load(r, Loader=yaml.Loader)

# duration = 0
# for x in y:
#     duration += x['duration']
# duration / 3600

In [47]:
# indices = list(range(train_df.shape[0]))
# random.shuffle(indices)
# save_df_to_tsv(train_df.iloc[indices[:10000]], os.path.join(root, 'train-tiny_asr.tsv'))

In [48]:
save_dir = os.path.join(root, 'en-{}'.format(lang), 'data', split, 'align_sllama')
# save_dir = os.path.join(root, 'mt-en', 'data', 'asr', 'align_mfat')
# save_dir = os.path.join(root, 'data', split, 'align_sllama')

In [49]:
os.makedirs(save_dir, exist_ok=True)

In [50]:
save_dir

'/mnt/data/siqiouyang/datasets/must-c-v1.0/en-es/data/train/align_sllama'

In [35]:
last_audio_path = None
for idx in tqdm(range(len(df))):
    audio_path, offset, num_frames = os.path.join(root, df['audio'][idx]).split(':')
    # audio_path = os.path.join(root, df['audio'][idx])
    # offset, num_frames = 0, df['n_frames'][idx]
    offset, num_frames = int(offset), int(num_frames)
    if last_audio_path is None or audio_path != last_audio_path:
        waveform, frame_rate = torchaudio.load(os.path.join(root, audio_path))
        last_audio_path = audio_path
    torchaudio.save(os.path.join(save_dir, '{}.wav'.format(df['id'][idx])), waveform[:, offset : offset + num_frames], sample_rate=frame_rate)

100%|██████████| 1312/1312 [00:03<00:00, 414.78it/s]


In [51]:
sentences = df['src_text'].tolist()

In [52]:
# punctuation = '!"#$%&,.?' # for maltese

In [53]:
def covered(s, punctuation):
    for c in s:
        if c not in punctuation:
            return False
    return True

space = '▁'
tokenized_sentences = []
segmentss = []
punctuation = punctuation
for sent in tqdm(df['src_text'].tolist()):
    # tokens = spm.EncodeAsPieces(sent)
    tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(sent, add_special_tokens=False))
    segments = []
    last = -1
    for idx, token in enumerate(tokens):
        if token.startswith(space) or covered(token, punctuation):
            if last != -1 and last <= idx - 1:
                segments.append((last, idx - 1))
            last = idx + (token == space or covered(token, punctuation) or \
                (token.startswith(space) and len(token) > 1 and covered(token[1:], punctuation)))    
    
    if last < len(tokens):
        segments.append((last, len(tokens) - 1))

    tokenized_sentence = []
    for seg in segments:
        token = ''.join(tokens[seg[0] : seg[1] + 1]).replace(space, '')
        if token.replace(',', '').isnumeric():
            token = token.replace(',', '')
        tokenized_sentence.append(token)

    tokenized_sentences.append(tokenized_sentence)
    segmentss.append(segments)

  0%|          | 318/260041 [00:00<01:21, 3173.10it/s]

100%|██████████| 260041/260041 [01:08<00:00, 3788.12it/s]


In [37]:
for i, id in enumerate(tqdm(df['id'])):
    # if id.startswith('common_voice'):
    #     write_dir = os.path.join(save_dir, df.iloc[i]['speaker'])
    # else:
    #     write_dir = os.path.join(save_dir, id[:id.rfind('_')])
    write_dir = save_dir 
    with open(os.path.join(write_dir, '{}.txt'.format(id)), 'w') as w:
        w.write(' '.join(tokenized_sentences[i]))

  0%|          | 0/1312 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/siqiouyang/datasets/must-c-v1.0/en-es/data/dev/align_sllama/ted_767/ted_767_0.txt'

```bash
mfa align --clean . english_mfa english_mfa textgrids 
mfa train --clean -o model/acoustic_model --phone_set IPA --output_format long_textgrid --include_original_text -t /mnt/data/siqiouyang/cache/MFA/mfat_enes ./ english_mfa ./textgrids 

mfa train --clean -o model/acoustic_model --phone_set IPA --output_format long_textgrid --include_original_text -t /mnt/data/siqiouyang/cache/MFA/mfat_id ./ indonesian_cv ./textgrids --clean
mfa align . indonesian_cv model/acoustic_model textgrids

mfa train --clean -o model/acoustic_model --phone_set IPA --output_format long_textgrid --include_original_text -t /mnt/data/siqiouyang/cache/MFA/mfat_mt ./ maltese_cv ./textgrids -j 36
mfa align --clean . maltese_cv model/acoustic_model.zip textgrids 
```

In [55]:
filtered_grids = []
n_outlier = 0
sum_duration = 0.
ids = []
speech_word_col = []
text_word_col = []
for i, id in enumerate(tqdm(df['id'])):

    # subdir = df.iloc[i]['speaker'] if id.startswith('common_voice') else id[:id.rfind('_')]

    grid_path = os.path.join(save_dir, 'textgrids/{}.TextGrid'.format(id))

    if os.path.exists(grid_path):
        ids.append(id)
        grid = textgrids.TextGrid(grid_path)
        filtered_grid = [tok for tok in grid['words'] if tok.text != '']

        if len(filtered_grid) != len(tokenized_sentences[i]):
            # print(i, [w.text for w in filtered_grid], tokenized_sentences[i], sep='\n')
            n_outlier += 1
            speech_word_col.append(None)
            text_word_col.append(None)
            continue

        speech_word = [(word.xmin, word.xmax) for word in filtered_grid]
        text_word = segmentss[i]

        speech_word_col.append(speech_word)
        text_word_col.append(text_word)

        # audio_path = os.path.join(save_dir, '{}.wav'.format(id))
        # info = torchaudio.info(audio_path)
        # duration = info.num_frames / info.sample_rate
        # interval = interval / duration

        # sum_duration += duration

        # print(id, tokenized_sentences[i])

        # th.save([segmentss[i], interval], os.path.join(save_dir, '{}.pt'.format(id)))
    else:
        speech_word_col.append(None)
        text_word_col.append(None)

100%|██████████| 260041/260041 [07:24<00:00, 584.73it/s] 


In [56]:
df['speech_word'] = speech_word_col
df['text_word'] = text_word_col

In [57]:
df

,id,audio,n_frames,speaker,src_text,tgt_text,src_lang,tgt_lang,speech_word,text_word
0,ted_1_0,en-es/data/train/wav/ted_1.wav:1739840:130720,130720,spk.1,(Laughter) I looked in the rear-view mirror an...,"Sé que suena como cualquier cosa para ustedes,...",en,es,"[(0.0, 0.27), (1.7, 1.89), (1.89, 2.17), (2.17...","[(1, 3), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9..."
1,ted_1_1,en-es/data/train/wav/ted_1.wav:1872800:99679,99679,spk.1,There was no motorcade back there. (Laughter) ...,No había caravana de vehículos allá atrás. ¿Ha...,en,es,"[(0.0, 0.13), (0.13, 0.31), (0.31, 0.47), (0.4...","[(0, 0), (1, 1), (2, 2), (3, 4), (5, 5), (6, 6..."
2,ted_1_2,en-es/data/train/wav/ted_1.wav:2163679:74880,74880,spk.1,"(Laughter) It was dinnertime, and we started l...","Era la hora de la cena, y comenzamos a buscar ...",en,es,"[(0.0, 0.27), (0.27, 0.37), (0.37, 0.71), (0.7...","[(1, 3), (5, 5), (6, 6), (7, 10), (12, 12), (1..."
3,ted_1_3,en-es/data/train/wav/ted_1.wav:2254400:8800,8800,spk.1,We were on I-40.,Estábamos en la I-40.,en,es,"[(0.0, 0.06), (0.06, 0.22), (0.22, 0.33), (0.3...","[(0, 0), (1, 1), (2, 2), (3, 3), (5, 6)]"
4,ted_1_4,en-es/data/train/wav/ted_1.wav:2271040:51840,51840,spk.1,"We got to Exit 238, Lebanon, Tennessee.","Llegamos a la Salida 238, Líbano, Tennessee.",en,es,"[(0.0, 0.04), (0.04, 0.31), (0.31, 1.09), (1.1...","[(0, 0), (1, 1), (2, 2), (3, 3), (5, 7), (9, 1..."
...,...,...,...,...,...,...,...,...,...,...
260036,ted_13591_40,en-es/data/train/wav/ted_13591.wav:851680:126240,126240,spk.13591,Then I would beat my plate with a spoon and my...,Luego golpeaba mi plato con una cuchara y me b...,en,es,"[(0.0, 0.63), (1.57, 1.7), (1.7, 1.98), (1.98,...","[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5..."
260037,ted_13591_41,en-es/data/train/wav/ted_13591.wav:988800:172160,172160,spk.13591,"I have been beating the same plates, shakers, ...",Desde entonces sigo golpeando los mismos plato...,en,es,"[(0.03, 0.15), (0.15, 0.38), (0.38, 0.55), (0....","[(0, 0), (1, 1), (2, 2), (3, 4), (5, 5), (6, 6..."
260038,ted_13591_43,en-es/data/train/wav/ted_13591.wav:1263840:96160,96160,spk.13591,"As I grew up, subconsciously, I felt a strong ...","A medida que iba creciendo, sentía inconscient...",en,es,"[(0.03, 0.15), (0.15, 0.47), (0.5, 0.72), (0.7...","[(0, 0), (1, 1), (2, 2), (3, 3), (5, 8), (10, ..."
260039,ted_13591_44,en-es/data/train/wav/ted_13591.wav:1363680:91520,91520,spk.13591,"Even without it being said out loud, I knew th...","Y aunque no lo decía en voz alta, sabía que es...",en,es,"[(0.03, 0.3), (0.3, 0.63), (0.63, 0.78), (0.78...","[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5..."


In [60]:
save_df_to_tsv(df, os.path.join(root, '{}_st_{}_mfa.tsv'.format(split, lang)))

In [107]:
df['n_frames'].sum() / 16000 / 3600

9.324904479166667

In [108]:
sum_duration / 3600

5.026304687499981

In [110]:
n_outlier / len(df)

0.1889837289943985

In [17]:
filtered_grids = []
n_outlier = 0
for i, id in enumerate(tqdm(df['id'])):
    grid_path = os.path.join(save_dir, 'textgrids/{}.TextGrid'.format(id))
    if os.path.exists(grid_path):
        grid = textgrids.TextGrid(grid_path)
        filtered_grid = [tok for tok in grid['words'] if tok.text != '']

        if len(filtered_grid) != len(tokenized_sentences[i]):
            # print(i, [w.text for w in filtered_grid], tokenized_sentences[i], sep='\n')
            n_outlier += 1
            continue

        interval = np.array([(word.xmin, word.xmax) for word in filtered_grid])
        audio_path = os.path.join(save_dir, '{}.wav'.format(id))
        info = torchaudio.info(audio_path)
        duration = info.num_frames / info.sample_rate
        interval = interval / duration

        os.system('cp {} ./'.format(os.path.join(save_dir, '{}.wav'.format(id))))
        print(sentences[i], tokenized_sentences[i], segmentss[i], interval)
        break

  0%|          | 0/8703 [00:00<?, ?it/s]

Jiena jogħġbuni wkoll. ['Jiena', 'jogħġbuni', 'wkoll'] [(0, 1), (2, 3), (4, 4)] [[0.         0.0304878 ]
 [0.0304878  0.95121951]
 [0.95121951 0.98780488]]


0.0

In [17]:
n_outlier

23

In [ ]:
filtered_grids = []
n_outlier = 0
for i, id in enumerate(tqdm(df['id'])):
    grid_path = os.path.join(save_dir, 'textgrids/{}.TextGrid'.format(id))
    if os.path.exists(grid_path):
        grid = textgrids.TextGrid(grid_path)

        phones = [phone.text if phone.text != '' else '<empty>' for phone in grid['phones']]

        interval = np.array([(phone.xmin, phone.xmax) for phone in grid['phones']])
        audio_path = os.path.join(save_dir, '{}.wav'.format(id))
        info = torchaudio.info(audio_path)
        duration = info.num_frames / info.sample_rate
        interval = interval / duration

        th.save([segmentss[i], interval], os.path.join(save_dir, '{}.phone.pt'.format(id)))

In [24]:
with open('/mnt/data/siqiouyang/datasets/must-c-v1.0/phone.txt', 'r') as r:
    all_phones = [p.strip() for p in r.readlines() if p.strip() != '']
    phone_dict = {p : i for i, p in enumerate(all_phones)}

In [5]:
g2p = G2p()
src_dict = PhonemeDictionary.load(os.path.join(root, 'phonemes.txt'))

In [6]:
for fn in os.listdir(root):
    # if fn.endswith('tsv') and ('asr' in fn or 'de.' in fn):
    if 'ls' in fn:

        df = load_df_from_tsv(os.path.join(root, fn))
        list_of_phonemes = []
        for src_text in tqdm(df['src_text'], desc=fn):
            raw_phonemes = g2p(src_text)
            phonemes = []
            for idx in range(len(raw_phonemes)):
                if raw_phonemes[idx] in src_dict:
                    p = raw_phonemes[idx]
                    if idx == 0 or raw_phonemes[idx - 1] not in src_dict:
                        p = '▁' + p
                    phonemes.append(p)
            list_of_phonemes.append(' '.join(phonemes))
        df['src_phoneme'] = list_of_phonemes
        save_df_to_tsv(df, os.path.join(root, fn))

dev_ls960_asr.tsv: 100%|██████████| 5567/5567 [00:07<00:00, 721.19it/s]


In [4]:
# For Librispeech

df = load_df_from_tsv('/mnt/data/siqiouyang/datasets/must-c-v1.0/dev_ls960_asr.tsv')

def covered(s, punctuation):
    for c in s:
        if c not in punctuation:
            return False
    return True

space = '▁'
tokenized_sentences = []
segmentss = []
punctuation = punctuation + '—’'
punctuation = punctuation.replace("'", '')
for sent in tqdm(df['src_text'].tolist()):
    tokens = spm.EncodeAsPieces(sent)
    segments = []
    last = -1
    for idx, token in enumerate(tokens):
        if token.startswith(space) or covered(token, punctuation):
            if last != -1 and last <= idx - 1:
                segments.append((last, idx - 1))
            last = idx + (token == space or covered(token, punctuation) or \
                (token.startswith(space) and len(token) > 1 and covered(token[1:], punctuation)))    
    
    if last < len(tokens):
        segments.append((last, len(tokens) - 1))

    tokenized_sentence = []
    for seg in segments:
        token = ''.join(tokens[seg[0] : seg[1] + 1]).replace(space, '')
        if token.replace(',', '').isnumeric():
            token = token.replace(',', '')
        tokenized_sentence.append(token)

    tokenized_sentences.append(tokenized_sentence)
    segmentss.append(segments)

100%|██████████| 5567/5567 [00:00<00:00, 18686.26it/s]


In [5]:
filtered_grids = []
n_outlier = 0
iterator = tqdm(df['id'])
for i, id in enumerate(iterator):
    grid_path = os.path.join('/mnt/data/siqiouyang/datasets/librispeech/LibriSpeech/librispeech_mfa/textgrids/{}'.format(df['speaker'][i]), '{}.TextGrid'.format(id))
    if os.path.exists(grid_path):
        grid = textgrids.TextGrid(grid_path)
        filtered_grid = [tok for tok in grid['words'] if tok.text != '']

        u = v = 0
        intervals = []
        fail = False
        while u < len(filtered_grid) and v < len(tokenized_sentences[i]):
            if filtered_grid[u].text == tokenized_sentences[i][v]:
                intervals.append((filtered_grid[u].xmin, filtered_grid[u].xmax))
                u += 1
                v += 1
            elif tokenized_sentences[i][v].startswith(filtered_grid[u].text):
                if u < len(filtered_grid) - 1 and tokenized_sentences[i][v] == filtered_grid[u].text + filtered_grid[u + 1].text:
                    intervals.append((filtered_grid[u].xmin, filtered_grid[u + 1].xmax))
                    u += 2
                    v += 1
                else:
                    fail = True
                    break
            else:
                fail = True
                break
        
        if u < len(filtered_grid) or v < len(tokenized_sentences[i]):
            fail = True

        iterator.set_description('{:.2f}'.format(n_outlier / (i + 1)))

        if fail:
            # print(i, [w.text for w in filtered_grid], tokenized_sentences[i], sep='\n')
            # break
            n_outlier += 1
            continue
            

        interval = np.array(intervals)
        
        audio_path = os.path.join('/mnt/data/siqiouyang/datasets/librispeech', df['audio'][i])
        info = torchaudio.info(audio_path)
        duration = info.num_frames / info.sample_rate
        interval = interval / duration

        assert len(segmentss[i]) == len(interval)
        th.save([segmentss[i], interval], os.path.join('/mnt/data/siqiouyang/datasets/librispeech/LibriSpeech/librispeech_mfa/{}'.format(df['speaker'][i]), '{}.pt'.format(id)))
print(n_outlier / len(df))

0.00: 100%|██████████| 5567/5567 [00:06<00:00, 895.10it/s] 

0.0014370396982216634


In [8]:
interval

array([[0.03123891, 0.05679801],
       [0.05679801, 0.07880724],
       [0.10365637, 0.12992545],
       [0.12992545, 0.15264466],
       [0.15264466, 0.17607384],
       [0.18033369, 0.19027334],
       [0.19027334, 0.23358182],
       [0.3542776 , 0.38125666],
       [0.38125666, 0.40397586],
       [0.40397586, 0.43237487],
       [0.44373447, 0.46077387],
       [0.46645367, 0.49272275],
       [0.49272275, 0.50479233],
       [0.50479233, 0.51828186],
       [0.51828186, 0.53319134],
       [0.53319134, 0.58146965],
       [0.58146965, 0.60773873],
       [0.62193823, 0.63968761],
       [0.64820731, 0.66950657],
       [0.66950657, 0.68512602],
       [0.68512602, 0.68938587],
       [0.68938587, 0.70642528],
       [0.70642528, 0.73837416],
       [0.76038339, 0.79233227],
       [0.79233227, 0.80085197],
       [0.80085197, 0.8342208 ],
       [0.8342208 , 0.8455804 ],
       [0.8455804 , 0.87042953],
       [0.87042953, 0.90876819],
       [0.90876819, 0.91444799],
       [0.

# Combine Librispeech

In [65]:
root = '/mnt/data/siqiouyang/datasets/must-c-v1.0'
train_splits = ["train-clean-100", "train-clean-360", "train-other-500"]
dev_splits = ["dev-clean", "dev-other"]

In [67]:
dfs = []
for split in train_splits:
    df = load_df_from_tsv(os.path.join(root, split + '.tsv'))
    dfs.append(df)
combined_df = pd.concat(dfs)
save_df_to_tsv(combined_df, os.path.join(root, 'train_ls960_asr.tsv'))

In [72]:
dfs = []
for split in dev_splits:
    df = load_df_from_tsv(os.path.join(root, split + '.tsv'))
    dfs.append(df)
combined_df = pd.concat(dfs)
save_df_to_tsv(combined_df, os.path.join(root, 'dev_ls960_asr.tsv'))

# Produce Low Resource TSV

In [5]:
split = 'train'

In [6]:
df = load_df_from_tsv(os.path.join(root, '{}_st_{}.tsv'.format(split, lang)))

In [7]:
for duration in [0]: # in hours
    limit = duration * 60 * 60 * 16000
    indices = list(range(df.shape[0]))
    sel_mask = np.zeros((df.shape[0]), dtype=bool)
    if duration > 0:
        random.shuffle(indices)
        total = 0
        for i, idx in enumerate(indices):
            total += df['n_frames'][idx]
            sel_mask[idx] = True
            if total > limit:
                break
    st_df = df.iloc[sel_mask]
    asr_df = df.iloc[~sel_mask]

    # filter those without .pt files
    filter_mask = np.zeros((asr_df.shape[0]), dtype=bool)
    for i, id in enumerate(asr_df['id']):
        pt_path = os.path.join(root, 'en-{}'.format(lang), 'data/{}/align'.format(split), '{}.pt'.format(id))
        if not os.path.exists(pt_path):
            filter_mask[i] = True
    asr_df = asr_df.iloc[~filter_mask]

    if duration > 0:
        save_df_to_tsv(st_df, os.path.join(root, 'train-{}h_st.tsv'.format(duration)))
    save_df_to_tsv(asr_df, os.path.join(root, 'train-{}h_asr.tsv'.format(duration)))

In [8]:
df.shape

(225271, 8)

In [9]:
asr_df.shape

(215748, 8)

In [3]:
df = load_df_from_tsv(os.path.join(root, 'train-1h_asr.tsv'))

In [5]:
limit = 100 * 60 * 60 * 16000
indices = list(range(df.shape[0]))
sel_mask = np.zeros((df.shape[0]), dtype=bool)
random.shuffle(indices)
total = 0
for i, idx in enumerate(indices):
    total += df['n_frames'][idx]
    sel_mask[idx] = True
    if total > limit:
        break
asr_df = df.iloc[sel_mask]

In [6]:
asr_df['n_frames'].sum() / 16000 / 3600

100.00037677083333

In [7]:
save_df_to_tsv(asr_df, os.path.join(root, 'train-1h_asr_100h.tsv'))